In [2]:
# Importing required packages
import pandas as pd
import numpy as np


In [36]:
# Loading the json data into dataframe
jsondf = pd.read_json('case.json')
print(type(jsondf))
print(jsondf.head())

<class 'pandas.core.frame.DataFrame'>
           EnqueuedTimeUtc            EventName  \
0  2021-09-05 08:04:08 UTC  DynamicPrice_Result   
1  2021-08-18 11:43:23 UTC  DynamicPrice_Result   
2  2021-09-05 09:04:04 UTC  DynamicPrice_Result   
3  2021-08-25 05:02:55 UTC   CurateOffer_Result   
4  2021-09-05 08:03:28 UTC  DynamicPrice_Result   

                                             Payload  
0  {"provider":"ApplyDynamicPriceRange","offerId"...  
1  {"provider":"ApplyDynamicPricePerOption","offe...  
2  {"provider":"ApplyDynamicPriceRange","offerId"...  
3  [{"curationProvider":"ByPrice","offerId":"149f...  
4  {"provider":"ApplyDynamicPriceRange","offerId"...  


In [14]:
# how many rows and columns present in the dataframe
jsondf.shape

(37, 3)

In [40]:

# Converting the EnqueuedTimeUtc to Brazillian time zone
jsondf["EnqueuedTimeUtc"] = pd.to_datetime(arg = jsondf["EnqueuedTimeUtc"]).dt.tz_convert("Brazil/East")

jsondf.head()


,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


In [46]:
# counting the distriv=bution based on eventname
jsondf["EventName"].value_counts()

EventName
DynamicPrice_Result    32
CurateOffer_Result      5
Name: count, dtype: int64

In [52]:
# Selecting DynamicPriceresult event for parsing
dynamicpriceropt = jsondf[jsondf["EventName"]=="DynamicPrice_Result"]
dynamicpriceropt.shape

(32, 3)

In [54]:
dynamicpriceropt.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 08:44:00-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [56]:
# Parsing the DynamicPricePerOption into strcutured data
import json


def dynamic_price_option(row):
    """Transform the JSON payload into a dynamic price option dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPricePerOption":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        for unique_option in payload_row["algorithmOutput"]:
            price_option = {
                "Provider": provider,
                "OfferId": offer_id,
                "UniqueOptionId": f"\"{unique_option['uniqueOptionId']}\"",
                "BestPrice": unique_option["bestPrice"],
                "EnqueuedTimeSP": enqueued_time
            }
            result.append(price_option)
    return result


lst = dynamicpriceropt.apply(dynamic_price_option, axis=1)
flatten = [x for xs in lst for x in xs]
dynamic_price_option_payload = pd.DataFrame(flatten)

In [59]:
# Checking the data after parsing
dynamic_price_option_payload

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP
0,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
1,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21
2,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""",92.45,18/08/21
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.60,18/08/21
4,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.60,18/08/21
5,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b7a7b6d1-4dae-7392-5aaf-f3369c29db1d""",93.00,18/08/21
6,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""577e4bbd-f49d-ac23-56a6-e70072a05229""",93.00,18/08/21
7,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""f9b876ab-2590-952f-d69d-5b352ec251f3""",91.35,18/08/21
8,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
9,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21


In [63]:
# Saving the parsed data into csv file
dynamic_price_option_payload.to_csv("DynamicPriceOption.csv", index=False)

In [65]:
# Parsing the DynamicPriceRange into strcutured data

def dynamic_price_range(row):
    """Transform the JSON payload into a dynamic price range dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPriceRange":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        algorithm_output = payload_row["algorithmOutput"]
        price_range = {
            "Provider": provider,
            "OfferId": offer_id,
            "MinGlobal": algorithm_output["min_global"],
            "MinRecommended": algorithm_output["min_recommended"],
            "MaxRecommended": algorithm_output["max_recommended"],
            "DifferenceMinRecommendMinTheory": algorithm_output["differenceMinRecommendMinTheory"],
            "EnqueuedTimeSP": enqueued_time
        }
        result.append(price_range)
    return result


lst = dynamicpriceropt.apply(dynamic_price_range, axis=1)
flatten = [x for xs in lst for x in xs]
dynamic_price_range_payload = pd.DataFrame(flatten)

In [67]:
dynamic_price_range_payload

,Provider,OfferId,MinGlobal,MinRecommended,MaxRecommended,DifferenceMinRecommendMinTheory,EnqueuedTimeSP
0,"""ApplyDynamicPriceRange""","""a6611d55-9624-4381-8cdd-323ee3689241""",85.00,87.20,97.65,2.2,05/09/21
1,"""ApplyDynamicPriceRange""","""b8c636fa-8241-47dc-ac40-bdf438a04d9c""",85.00,87.20,97.65,2.2,05/09/21
2,"""ApplyDynamicPriceRange""","""3d32f7fb-396d-4d3f-b673-dea1f7dc41b7""",85.00,87.20,97.65,2.2,05/09/21
3,"""ApplyDynamicPriceRange""","""329194f3-95a4-45ef-b3d0-2796f74ce2a0""",85.00,87.20,97.65,2.2,05/09/21
4,"""ApplyDynamicPriceRange""","""fdcfde5c-113d-4a59-9ae0-8bc31e2943d8""",87.35,89.25,99.95,1.9,05/09/21
5,"""ApplyDynamicPriceRange""","""27bbc4fa-2388-4780-b66c-92a51397d191""",87.35,89.25,99.95,1.9,05/09/21
6,"""ApplyDynamicPriceRange""","""baffc30b-7642-45fe-a2ce-da31a71732ae""",85.00,87.20,97.65,2.2,05/09/21
7,"""ApplyDynamicPriceRange""","""b5982abd-f602-47ac-b45a-bb43bf993d46""",85.00,87.20,97.65,2.2,05/09/21
8,"""ApplyDynamicPriceRange""","""f6643886-4a0f-45ae-ae32-ee95c72ee94a""",87.35,89.25,99.95,1.9,05/09/21
9,"""ApplyDynamicPriceRange""","""16a2d492-b1c3-40ec-970f-b8704d8db96f""",85.00,87.20,97.65,2.2,05/09/21


In [69]:
# Saving the parsed data into csv file
dynamic_price_option_payload.to_csv("DynamicPriceRange.csv", index=False)

In [73]:
# Selecting CurateOffer_Result event for parsing

curated_offers = jsondf[jsondf["EventName"] == "CurateOffer_Result"]
curated_offers.shape

(5, 3)

In [81]:
curated_offers.head(10)

,EnqueuedTimeUtc,EventName,Payload
3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
13,2021-08-25 06:03:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""c99a..."
15,2021-08-25 02:02:18-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""d668..."
17,2021-08-25 06:02:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""0a06..."
35,2021-08-25 06:03:14-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""135d..."


In [77]:
# Parsing the curationProvider into strcutured data


def curated_offer(row):
    """Transform JSON payload into curated offer dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    for el in payload_row:
        curation_provider = f"\"{el['curationProvider']}\""
        offer_id = f"\"{el['offerId']}\""
        dealer_id = f"\"{el['dealerId']}\""
        for option in el["options"]:
            offer_options = {
                "CurationProvider": curation_provider,
                "OfferId": offer_id,
                "DealerId": dealer_id,
                "UniqueOptionId": f"\"{option['uniqueOptionId']}\"",
                "OptionId": f"\"{option['optionId']}\"",
                "IsMobileDealer": option["isMobileDealer"],
                "IsOpen": option["isOpen"],
                "Eta": f"\"{option['eta']}\"",
                "ChamaScore": option["chamaScore"],
                "ProductBrand": f"\"{option['productBrand']}\"",
                "IsWinner": option["isWinner"],
                "MinimumPrice": option["minimumPrice"],
                "MaximumPrice": option["maximumPrice"],
                "DynamicPrice": option["dynamicPrice"],
                "FinalPrice": option["finalPrice"]
            }
            if "defeatPrimaryReason" in option:
                offer_options["DefeatPrimaryReason"] = f"\"{option['defeatPrimaryReason']}\""
                offer_options["DefeatReasons"] = f"\"{option['defeatReasons']}\""
            else:
                offer_options["DefeatPrimaryReason"] = "\"\""
                offer_options["DefeatReasons"] = "\"\""
            offer_options["EnqueuedTimeSP"] = enqueued_time
            result.append(offer_options)
    return result


flatten = [x for xs in curated_offers.apply(curated_offer, axis=1) for x in xs]
curated_offers_payload = pd.DataFrame(flatten)

In [79]:
curated_offers_payload

,CurationProvider,OfferId,DealerId,UniqueOptionId,OptionId,IsMobileDealer,IsOpen,Eta,ChamaScore,ProductBrand,IsWinner,MinimumPrice,MaximumPrice,DynamicPrice,FinalPrice,DefeatPrimaryReason,DefeatReasons,EnqueuedTimeSP
0,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""b0e296a9-0590-f0e0-8211-243a2ededb12""","""6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2""",True,True,"""1:00""",8.0,"""ULTRAGAZ""",True,90.00,180.00,91.90,91.90,"""""","""""",25/08/21
1,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""","""6517 || 6517""",False,False,"""0:01""",8.0,"""ULTRAGAZ""",False,90.00,180.00,91.90,91.90,"""Closed""","""['Closed', 'HasDriverInOffer']""",25/08/21
2,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""","""9047 || 9047 || ULTRAGAZ""",False,False,"""1:00""",9.0,"""ULTRAGAZ""",False,99.00,198.00,99.95,99.95,"""Closed""","""['Closed', 'HigherPrice', 'HasDriverInOffer']""",25/08/21
3,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""","""9047 || 9047 || CONSIGAZ""",False,False,"""1:00""",9.0,"""CONSIGAZ""",False,89.99,179.98,91.89,91.89,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/21
4,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""577e4bbd-f49d-ac23-56a6-e70072a05229""","""9047 || 9047 || LIQUIGAS""",False,False,"""1:00""",9.0,"""LIQUIGAS""",False,92.00,184.00,93.90,93.90,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/21
5,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""f9b876ab-2590-952f-d69d-5b352ec251f3""","""9047 || 9047 || NACIONALGAS""",False,False,"""1:00""",9.0,"""NACIONALGAS""",True,88.90,177.80,90.80,90.80,"""""","""""",25/08/21
6,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""8330""","""151e59ac-761a-96f5-d2b9-882037a9fd28""","""8330 || 8330 || CONSIGAZ""",False,True,"""0:01""",8.0,"""CONSIGAZ""",True,80.00,160.00,87.35,87.35,"""""","""""",25/08/21
7,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""8835""","""b7a7b6d1-4dae-7392-5aaf-f3369c29db1d""","""8835 || 8835 || LIQUIGAS""",False,False,"""0:01""",8.0,"""LIQUIGAS""",True,60.00,120.00,87.35,87.35,"""""","""""",25/08/21
8,"""ByPrice""","""c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d""","""6517""","""b0e296a9-0590-f0e0-8211-243a2ededb12""","""6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2""",True,True,"""00:12:54.9215999""",8.0,"""ULTRAGAZ""",True,90.00,180.00,91.90,91.90,"""""","""""",25/08/21
9,"""ByPrice""","""c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d""","""6517""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""","""6517 || 6517""",False,True,"""0:01""",8.0,"""ULTRAGAZ""",False,90.00,180.00,91.90,91.90,"""HasDriverInOffer""","""['HasDriverInOffer']""",25/08/21


In [83]:
# Saving the parsed data into csv file

curated_offers_payload.to_csv("CuratedOfferOptions.csv", index=False)